In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import re
import pandas as pd 

In [3]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 스크롤 로직
def scroll_to_bottom(scrollable_element):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        time.sleep(1)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

# 네이버 지도 접속
driver.get("https://map.naver.com/")
time.sleep(3)

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
)
search_box.send_keys("구로구 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# iframe 전환
try:
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
    )
    driver.switch_to.frame(iframe)
except Exception as e:
    print(f"iframe 전환 오류: {e}")
    driver.quit()
    exit()

# 결과 리스트 초기화
results = []
visited_urls = set()
page_number = 1
MAX_PAGES = 5  # 최대 페이지 수
MAX_RESULTS = 150  # 최대 결과 수 제한

while page_number <= MAX_PAGES and len(results) < MAX_RESULTS:
    try:
        # 스크롤을 맨 아래까지 내리기
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        scroll_to_bottom(scrollable_element)

        # 현재 페이지의 가게 리스트 가져오기
        store_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "place_bluelink"))
        )
        print(f"현재 {page_number}페이지) 총 {len(store_elements)}개의 가게를 찾았습니다.")

        for store in store_elements:
            if len(results) >= MAX_RESULTS:
                print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
                break

            try:
                store.click()
                time.sleep(random.uniform(3, 5))

                # 상세 iframe 전환
                driver.switch_to.default_content()
                detail_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                driver.switch_to.frame(detail_iframe)

                # 페이지 소스 파싱
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                # 가게 정보 추출
                place_name = soup.select_one("span.GHAhO").text.strip() if soup.select_one("span.GHAhO") else "N/A"
                category = soup.select_one("span.lnJFt").text.strip() if soup.select_one("span.lnJFt") else "N/A"
                address = soup.select_one("span.LDgIH").text.strip() if soup.select_one("span.LDgIH") else "N/A"
                current_url = driver.current_url
                res_code = re.findall(r"place/(\d+)", current_url)
                store_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#' if res_code else "N/A"

                # 중복 여부 확인
                if store_url not in visited_urls:
                    visited_urls.add(store_url)
                    results.append({
                        "가게 이름": place_name,
                        "업종구분": category,
                        "주소": address,
                        "URL": store_url
                    })
                    print(f"{len(results)}. {place_name}")

                # iframe 복귀
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"가게 크롤링 중 오류 발생: {e}")

        if len(results) >= MAX_RESULTS:
            print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
            break

        # '다음' 버튼 클릭
        if page_number < MAX_PAGES:
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//a[@class="eUTV2" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", next_button)
                print(f"'다음' 버튼 클릭: 페이지 {page_number + 1}로 이동 중")
                time.sleep(random.uniform(5, 10))

                # iframe 전환
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"'다음' 버튼 클릭 중 오류 발생: {e}")
                break

        # 페이지 번호 증가
        page_number += 1

    except Exception as e:
        print(f"페이지 크롤링 중 오류 발생: {e}")
        break

# CSV 파일 저장
file_name = "Guro-gu_restaurants.csv"
df = pd.DataFrame(results)
df.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"크롤링 완료. 결과는 {file_name}에 저장되었습니다.")
driver.quit()

현재 1페이지) 총 76개의 가게를 찾았습니다.
1. 영등포구석집 신도림점
2. 육연사 개봉역점
3. 봉고기
4. 육풍 구로디지털단지점
5. 오목집 신도림점
6. 세광양대창 구로디지털점
7. 할매기름집
8. 우월소곱창 신도림본점
9. 신도림 이도식당
10. 크랩프트 신도림점
11. 옛날마차
12. 미켈고깃집 구로점
13. 새벽집 양곱창
14. 거북솥삼겹살
15. 엉베흐 신도림
16. 교대이층집 신도림점
17. 미방 신도림점
18. 라도맨션
19. 한사발포차 신도림점
20. 라무진 신도림역점
21. 계절별미
22. 라마노피자
23. 의영이네 양갈비
24. 한수한식
25. 청록미나리식당 고척목동점
26. 아건구디역점
27. 오동
28. 한짬뽕
29. 월래순교자관
30. 광명수산 신도림역점
31. 하나마토 신도림본점
32. 내고향 소머리국밥
33. 옛날황소곱창
34. 참토우 구로디지털단지 10호점
35. 샤브몰 서울구로구청점
36. 문어부인삼교비
37. 라그릴리아 신도림디큐브시티점
38. 본가 구로디지털단지역점
39. 중화백반
40. 월화식당 구로디지털단지역점
41. 메이비카페
42. 옛날마차신관
43. 야키토리잔잔 구로디지털역
44. 바르미 스시샤브 현대백화점디큐브시티점
45. 제줏간 구로디지털단지점
46. 성공관
47. 갓파스시 구로점
48. 해탄 구로디지털단지역점
49. 청이
50. 우이락 신도림점
51. 교대이층집 구로디지털단지점
52. 석쇳집
53. 신림춘천집 구로디지털 직영점
54. 5.13낙곱새 구로디지털직영점
55. 스시메이진 구로
56. 낙원갈비집 구로구청점
57. 부대찌개대사관 구로디지털본점
58. 최우영스시
59. 닥터로빈 고척점
60. 오사카로
61. 상록회관연탄구이 개봉역점
62. 포포인츠 바이 쉐라톤 구로 테이블 32 뷔페 레스토랑
63. 마장동김씨 구로디지털점
64. 애슐리퀸즈 NC신구로점
65. 옹기종기감자옹심이
66. 지구쓸맥 구로디지털점
67. 사계진미숯불닭갈비 구로디지털점
68. 강촌숯불닭갈비 본점
69. 이화참숯갈비
70. 정통집